<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
# 📘 PySpark - expr() - ביטויים דמויי SQL
[PySpark SQL expr() (Expression) Function](https://sparkbyexamples.com/pyspark/pyspark-sql-expr-expression-function/)
## תוכן עניינים
1. [הקדמה](#introduction)
2. [תחביר expr()](#syntax)
3. [שרשור עמודות עם || (concat)](#concat)
4. [CASE WHEN עם expr()](#case-when)
5. [שימוש בערך קיים מעמודה](#column-value)
6. [מתן כינוי (alias) לעמודה](#alias)
7. [המרת טיפוסים (cast)](#cast)
8. [פעולות חשבון](#arithmetic)
9. [סינון נתונים (filter)](#filter)
10. [דוגמה מלאה](#complete-example)
11. [מקורות](#references)

## 🎯 הקדמה <a id='introduction'></a>

**PySpark expr()** היא פונקציית SQL המאפשרת להריץ ביטויים דמויי SQL ולהשתמש בערך קיים מעמודת DataFrame כארגומנט לביטוי.

רוב הפונקציות הנפוצות ב-SQL הן חלק מהמחלקה `pyspark.sql.Column` או מה-API של `pyspark.sql.functions`. בנוסף לאלו, PySpark תומכת גם בפונקציות SQL רבות נוספות, ולכן כדי להשתמש בהן צריך להשתמש בפונקציה `expr()`.

### שני מקרי שימוש עיקריים:
1. **שימוש בפונקציות שאינן קיימות ב-`PySpark Column`** (למשל: `CASE WHEN`, `regexp_count()`)
2. **הרחבת הפונקציונליות של PySpark SQL** על ידי אפשרות להשתמש בעמודות DataFrame בתוך ביטויים

## 📦 ייבוא ספריות והכנת נתונים

ראשית, נייבא את הספריות הנדרשות וניצור DataFrame לדוגמה:

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

# יצירת DataFrame עם נתוני שמות
data = [
    ("James", "Bond"),
    ("Scott", "Varsa")
]

df = spark.createDataFrame(data=data, schema=["col1", "col2"])
df.show()

## 📝 1. תחביר expr() <a id='syntax'></a>

```python
expr(str)
```

הפונקציה `expr()` מקבלת ביטוי SQL כארגומנט מסוג מחרוזת, מבצעת את הביטוי, ומחזירה אובייקט מסוג `pyspark.sql.Column`.

ביטויים שסופקו עם הפונקציה הזו אינם מהווים בטיחות compile-time כמו פעולות DataFrame רגילות.

## 🔗 2. שרשור עמודות עם || (דומה ל-SQL) <a id='concat'></a>

אם יש לך רקע ב-SQL, אתה בוודאי מכיר את השימוש ב-`||` לשרשור ערכים משתי עמודות מחרוזת. אפשר לעשות בדיוק את אותו הדבר עם `expr()`:

In [ ]:
from pyspark.sql.functions import expr

# שרשור שמות
data = [("James", "Bond"), ("Scott", "Varsa")]
df = spark.createDataFrame(data=data, schema=["col1", "col2"])

df.withColumn("Name", expr("col1 || ',' || col2")).show()

## ⚖️ 3. שימוש ב-CASE WHEN עם expr() <a id='case-when'></a>

**PySpark לא תומך ב-CASE WHEN בצורה ישירה** דרך `withColumn()` או `select()`. על מנת להשתמש בביטוי CASE WHEN, צריך להשתמש ב-`expr()`.

**חלופה:** אפשר גם להשתמש ב-`when().otherwise()` במקום.

In [ ]:
# יצירת DataFrame עם נתוני מגדר
data = [("James", "M"), ("Michael", "F"), ("Jen", "")]
columns = ["name", "gender"]
df = spark.createDataFrame(data=data, schema=columns)

# שימוש ב-CASE WHEN כדי להחליף ערכים
df2 = df.withColumn(
    "gender",
    expr(
        "CASE WHEN gender = 'M' THEN 'Male' " +
        "WHEN gender = 'F' THEN 'Female' " +
        "WHEN gender IS NULL THEN '' " +
        "ELSE gender END"
    )
)

df2.show()

## 💡 4. שימוש בערך קיים מעמודה אחרת <a id='column-value'></a>

רוב הפונקציות לוקחות ערכי קבועים (literal), אבל לפעמים אנחנו צריכים להשתמש בערך מעמודה קיימת במקום קבוע. הנה דוגמה שמוסיפה ערך חודשים מעמודה קיימת:

In [ ]:
# הוספת חודשים מעמודה
data = [
    ("2019-01-23", 1),
    ("2019-06-24", 2),
    ("2019-09-20", 3)
]

df = spark.createDataFrame(data=data, schema=["date", "increment"])

df.select(
    df.date,
    df.increment,
    expr("add_months(date, increment) as inc_date")
).alias("inc_date").show()

**הערה:** ייבוא פונקציות SQL לא נדרש כאשר משתמשים בהן עם `expr()`. רואים למעלה ש-`add_months()` נעשה בלי לייבא.

## 🏷️ 5. מתן כינוי (alias) לעמודה באמצעות 'as' <a id='alias'></a>

אפשר לספק כינוי לעמודה באמצעות המילה 'as' בתוך הביטוי של `expr()`:

In [ ]:
# מתן כינוי עם 'as'
df.select(
    df.date,
    df.increment,
    expr("""add_months(date, increment) as inc_date""")
).show()

## 🔄 6. המרת טיפוסים עם cast() <a id='cast'></a>

הדוגמה הבאה ממירה טיפוס long לטיפוס String:

In [ ]:
# המרת טיפוס
df.select(
    "increment",
    expr("cast(increment as string) as str_increment")
).printSchema()

## ➕ 7. פעולות חשבון <a id='arithmetic'></a>

`expr()` משמש גם לביצוע פעולות חשבון. הדוגמה הבאה מוסיפה 5 לערך `increment` ויוצרת עמודה חדשה `new_increment`:

In [ ]:
# פעולות חשבון
df.select(
    df.date,
    df.increment,
    expr("increment + 5 as new_increment")
).show()

## 🔍 8. סינון שורות עם filter() <a id='filter'></a>

אפשר להשתמש ב-`expr()` גם לסינון שורות של DataFrame:

In [ ]:
# סינון נתונים
data = [(100, 2), (200, 3000), (500, 500)]
df = spark.createDataFrame(data=data, schema=["col1", "col2"])

df.filter(expr("col1 == col2")).show()

## 🎓 9. דוגמה מלאה <a id='complete-example'></a>

הנה דוגמה מקיפה המשלבת את כל מה שלמדנו:

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

# שרשור עמודות
data = [("James", "Bond"), ("Scott", "Varsa")]
df = spark.createDataFrame(data=data, schema=["col1", "col2"])
df.withColumn("Name", expr("col1 || ',' || col2")).show()

# שימוש ב-CASE WHEN
data = [("James", "M"), ("Michael", "F"), ("Jen", "")]
columns = ["name", "gender"]
df = spark.createDataFrame(data=data, schema=columns)
df2 = df.withColumn(
    "gender",
    expr("CASE WHEN gender = 'M' THEN 'Male' " +
         "WHEN gender = 'F' THEN 'Female' " +
         "WHEN gender IS NULL THEN '' " +
         "ELSE gender END")
)
df2.show()

# הוספת חודשים מערך בעמודה
data = [("2019-01-23", 1), ("2019-06-24", 2), ("2019-09-20", 3)]
df = spark.createDataFrame(data=data, schema=["date", "increment"])
df.select(
    df.date,
    df.increment,
    expr("add_months(date, increment) as inc_date")
).show()

# מתן כינוי
df.select(
    df.date,
    df.increment,
    expr("""add_months(date, increment) as inc_date""")
).show()

# פעולות חשבון
df.select(
    df.date,
    df.increment,
    expr("increment + 5 as new_increment")
).show()

# המרת טיפוסים
df.select(
    "increment",
    expr("cast(increment as string) as str_increment")
).printSchema()

# סינון
data = [(100, 2), (200, 3000), (500, 500)]
df = spark.createDataFrame(data=data, schema=["col1", "col2"])
df.filter(expr("col1 == col2")).show()

## ✨ סיכום

הפונקציה `expr()` ב-PySpark מספקת דרך להריץ ביטויים דמויי SQL עם DataFrames, והיא מאפשרת:

- ✅ שימוש בפונקציות SQL שאינן זמינות ישירות ב-PySpark
- ✅ שרשור עמודות עם אופרטור `||`
- ✅ ביטויי CASE WHEN
- ✅ שימוש בערכים מעמודות אחרות בתוך ביטויים
- ✅ המרת טיפוסים, פעולות חשבון וסינון

**טיפ חשוב:** כאשר משתמשים ב-`expr()`, הביטויים אינם עוברים בדיקה בזמן קומפילציה, לכן יש להיזהר מטעויות כתיב!

## 📚 מקורות <a id='references'></a>

- [PySpark SQL Functions Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html)
- [SparkByExamples - PySpark expr() Function](https://sparkbyexamples.com/pyspark/pyspark-sql-expr-expression-function/)